In [22]:
import numpy as np
import random

def create_random_graph_adj_list(num_nodes=100, connectivity=0.1, min_weight=1, max_weight=10):
    graph = {i: [] for i in range(num_nodes)}
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < connectivity:
                weight = random.uniform(min_weight, max_weight)
                graph[i].append((j, weight))
                graph[j].append((i, weight))  # undirected graph
    return graph

def initialize_pheromone_adj_list(graph, initial_pheromone=1.0):
    pheromone = {}
    for node, edges in graph.items():
        pheromone[node] = {}
        for (neighbor, _) in edges:
            pheromone[node][neighbor] = initial_pheromone
    return pheromone

def compute_heuristic_adj_list(graph):
    heuristic = {}
    for node, edges in graph.items():
        heuristic[node] = {}
        for (neighbor, weight) in edges:
            heuristic[node][neighbor] = 1.0 / (weight + 1e-10)  # inverse weight
    return heuristic

def select_next_node_adj_list(current_node, visited, pheromone, heuristic, alpha, beta, graph):
    neighbors = graph[current_node]
    probabilities = []
    nodes = []
    for (neighbor, weight) in neighbors:
        if neighbor not in visited:
            tau = pheromone[current_node][neighbor] ** alpha
            eta = heuristic[current_node][neighbor] ** beta
            prob = tau * eta
            probabilities.append(prob)
            nodes.append(neighbor)

    if len(probabilities) == 0:
        return None  # dead end

    probabilities = np.array(probabilities)
    total = probabilities.sum()
    if total == 0:
        return None

    probabilities = probabilities / total
    return np.random.choice(nodes, p=probabilities)

def build_path_adj_list(graph, pheromone, heuristic, alpha, beta, source, destination):
    path = [source]
    current_node = source

    while current_node != destination:
        next_node = select_next_node_adj_list(current_node, path, pheromone, heuristic, alpha, beta, graph)
        if next_node is None:
            return None, float('inf')  # dead end
        path.append(next_node)
        current_node = next_node

    # Calculate path length
    length = 0
    for i in range(len(path) - 1):
        current = path[i]
        nxt = path[i + 1]
        # Find the weight between current and nxt
        for (neighbor, weight) in graph[current]:
            if neighbor == nxt:
                length += weight
                break

    return path, length

def update_pheromone_adj_list(pheromone, paths, evaporation_rate, Q=1.0):
    # Evaporate pheromone
    for node in pheromone:
        for neighbor in pheromone[node]:
            pheromone[node][neighbor] *= (1 - evaporation_rate)

    # Deposit pheromone
    for path, length in paths:
        if path is None or length == float('inf'):
            continue
        deposit_amount = Q / length
        for i in range(len(path) - 1):
            pheromone[path[i]][path[i + 1]] += deposit_amount

    return pheromone

def ant_colony_optimization_adj_list(graph, source, destination, num_ants=10, max_iter=100, alpha=1.0, beta=2.0, evaporation_rate=0.5, Q=1.0, max_stagnation=10):
    pheromone = initialize_pheromone_adj_list(graph)
    heuristic = compute_heuristic_adj_list(graph)

    best_path = None
    best_length = float('inf')
    stagnation_count = 0

    for iteration in range(max_iter):
        paths = []
        improved = False

        for _ in range(num_ants):
            path, length = build_path_adj_list(graph, pheromone, heuristic, alpha, beta, source, destination)
            paths.append((path, length))
            if length < best_length:
                best_length = length
                best_path = path
                improved = True

        pheromone = update_pheromone_adj_list(pheromone, paths, evaporation_rate, Q)

        print(f"Iteration {iteration + 1}: Best path length so far: {best_length}")

        if improved:
            stagnation_count = 0
        else:
            stagnation_count += 1

        if stagnation_count >= max_stagnation:
            print(f"Stopping early due to no improvement in last {max_stagnation} iterations.")
            break

    return best_path, best_length

def print_path_with_weights(graph, path):
    if path is None:
        print("No path found.")
        return
    print("Best path with weights:")
    for i in range(len(path) - 1):
        current = path[i]
        nxt = path[i + 1]
        weight = None
        for (neighbor, w) in graph[current]:
            if neighbor == nxt:
                weight = w
                break
        print(f"{current} --({weight:.2f})--> ", end="")
    print(path[-1])

# --- Example usage ---

num_nodes = 100
graph = create_random_graph_adj_list(num_nodes=num_nodes, connectivity=0.1)

source = 0
destination = num_nodes - 1

best_path, best_length = ant_colony_optimization_adj_list(
    graph, source, destination,
    num_ants=20,
    max_iter=100,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.5,
    Q=1.0,
    max_stagnation=5
)

print(f"\nFinal best path length: {best_length}")
print_path_with_weights(graph, best_path)


Iteration 1: Best path length so far: 9.209978096428554
Iteration 2: Best path length so far: 9.209978096428554
Iteration 3: Best path length so far: 9.209978096428554
Iteration 4: Best path length so far: 9.184766614276345
Iteration 5: Best path length so far: 9.184766614276345
Iteration 6: Best path length so far: 8.93631676816695
Iteration 7: Best path length so far: 8.93631676816695
Iteration 8: Best path length so far: 8.93631676816695
Iteration 9: Best path length so far: 8.93631676816695
Iteration 10: Best path length so far: 8.93631676816695
Iteration 11: Best path length so far: 8.93631676816695
Stopping early due to no improvement in last 5 iterations.

Final best path length: 8.93631676816695
Best path with weights:
0 --(6.90)--> 6 --(2.04)--> 99
